<a href="https://colab.research.google.com/github/jmcaussade/Homologacion-Car-Makers/blob/main/Pesos_dataset_github_frances.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
!pip install rapidfuzz
!pip install unidecode

In [90]:
import pandas as pd
import requests
import time
import re
from rapidfuzz.fuzz import ratio
from rapidfuzz import process
from unidecode import unidecode

In [91]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [92]:
from google.colab import files

In [131]:
ruta_data_set_maker_model = '/content/drive/MyDrive/Xinergy/Proyecto_homologacion_autos/Pesos/Trozo1.xlsx'
df_maker_model = pd.read_excel(ruta_data_set_maker_model, sheet_name='Sheet1')
df_maker_model.head()
df_maker_model

,Unnamed: 0,numcaso,tipo_vehiculo,fecha_servicio,plate,serie,model,Tipo de Auto,car_brand,car_model,...,Maker_api,Model_api,Type_api,Weight_api,clean_car_maker,clean_car_model,Maker_final,Maker_Similarity_score,Model_final,Model_Similarity_score
0,NaN,355845,Auto,2023-09-01 00:01:13,SZD861A,3HGCM66563G001615,2003,estandar,Honda,ACCORD,...,honda,accord,passenger car,"class 1c: 4,001 - 5,000 lb (1,814 - 2,268 kg)",honda,accord,honda,100.000000,accord,100
1,NaN,355847,Camion,2023-09-01 00:06:09,TP9498G,3N6DD25T2EK005444,2014,estandar,NISSAN,ESTACAS,...,NaN,NaN,truck,"class 1c: 4,001 - 5,000 lb (1,814 - 2,268 kg)",nissan,estacas,nissan,100.000000,estacas,0
2,NaN,355849,Auto,2023-09-01 00:08:38,NCZ1892,MEX5G2603JT120368,2018,estandar,Volkswagen,VENTO,...,NaN,NaN,NaN,NaN,volkswagen,vento,volkswagen,100.000000,vento,0
3,NaN,355857,Auto,2023-09-01 00:24:34,FCJ597B,1C4BJWEG6FL629655,2015,estandar,JEEP,J WRANGLER,...,jeep,wrangler,multipurpose passenger vehicle (mpv),"class 1d: 5,001 - 6,000 lb (2,268 - 2,722 kg)",jeep,j wrangler,jeep,100.000000,wrangler,100
4,NaN,355864,Auto,2023-09-01 00:46:09,NJA2747,MA6CB6CD6KT029772,2019,estandar,CHEVROLET,BEAT,...,NaN,NaN,NaN,NaN,chevrolet,beat,chevrolet,100.000000,beat,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,NaN,430448,Camion,2023-11-04 14:22:12,JX40237,3NFAB25R4MM000066,2021,estandar,HYUNDAI,CARGO VAN,...,NaN,NaN,NaN,NaN,hyundai,cargo van,hyundai,100.000000,cargo van,0
39996,NaN,430450,Auto,2023-11-04 14:23:14,JNA3243,9BWDB45U5HT015012,2017,estandar,Volkswagen,GOL,...,volkswagen,NaN,passenger car,NaN,volkswagen,gol,volkswagen,100.000000,gol,0
39997,NaN,430453,Auto,2023-11-04 14:25:29,TMH6032,WAUAYJ8V6KA077601,2019,estandar,Audi,A3,...,audi,NaN,passenger car,NaN,audi,a3,audi,100.000000,a3,0
39998,NaN,430455,Auto,2023-11-04 14:26:07,JSH4002,TMCJ33AB4MJ157474,2021,estandar,HYUNDAI,TUCSON,...,costin sports car,NaN,passenger car,NaN,hyundai,tucson,N/A caso 3,16.666667,tucson,0


In [132]:
# List of columns to keep
columns_to_keep = [
    "numcaso", "tipo_vehiculo", "fecha_servicio", "plate",
    "model", "Tipo de Auto", "car_type", "clean_serie", "Maker_final", "Model_final"
]

# Keeping only the specified columns
df_maker_model2 = df_maker_model[columns_to_keep]

# Display the filtered dataframe
df_maker_model2

,numcaso,tipo_vehiculo,fecha_servicio,plate,model,Tipo de Auto,car_type,clean_serie,Maker_final,Model_final
0,355845,Auto,2023-09-01 00:01:13,SZD861A,2003,estandar,Auto,3HGCM66563G001615,honda,accord
1,355847,Camion,2023-09-01 00:06:09,TP9498G,2014,estandar,camion,3N6DD25T2EK005444,nissan,estacas
2,355849,Auto,2023-09-01 00:08:38,NCZ1892,2018,estandar,Auto,MEX5G2603JT120368,volkswagen,vento
3,355857,Auto,2023-09-01 00:24:34,FCJ597B,2015,estandar,Auto,1C4BJWEG6FL629655,jeep,wrangler
4,355864,Auto,2023-09-01 00:46:09,NJA2747,2019,estandar,Auto,MA6CB6CD6KT029772,chevrolet,beat
...,...,...,...,...,...,...,...,...,...,...
39995,430448,Camion,2023-11-04 14:22:12,JX40237,2021,estandar,camion,3NFAB25R4MM000066,hyundai,cargo van
39996,430450,Auto,2023-11-04 14:23:14,JNA3243,2017,estandar,Auto,9BWDB45U5HT015012,volkswagen,gol
39997,430453,Auto,2023-11-04 14:25:29,TMH6032,2019,estandar,Auto,WAUAYJ8V6KA077601,audi,a3
39998,430455,Auto,2023-11-04 14:26:07,JSH4002,2021,estandar,Auto,TMCJ33AB4MJ157474,N/A caso 3,tucson


In [133]:
ruta_data_set_pesos = '/content/drive/MyDrive/Xinergy/Proyecto_homologacion_autos/Pesos/Preprocessed_data.xlsx'
df_pesos = pd.read_excel(ruta_data_set_pesos, sheet_name='Preprocessed_data')
df_pesos.head()
df_pesos

,Marca,Nombre del modelo,Modelo,Grupo,Descripción comercial,Energía,Carrocería,Cilindrada,Gama,Potencia fiscal,...,Consumo a velocidad media promedio,Consumo a alta velocidad promedio,Consumo a velocidad muy alta promedio,Consumo a velocidad mixta promedio,Consumo eléctrico promedio,CO2 a baja velocidad promedio,CO2 a velocidad media promedio,CO2 a alta velocidad promedio,CO2 a velocidad muy alta promedio,CO2 a velocidad mixta promedio
0,ALFA ROMEO,GIULIA,GIULIA,STELLANTIS,GIULIA 2.0 T Essence (200ch) AT8,ESSENCE,BERLINE,1995,SUPERIEURE,11,...,7.852000e+03,6.394000e+03,6789.0,7.440000e+16,-1.0,248405.0,177905.00,1.448750e+05,153825.00,168575.0
1,ALFA ROMEO,GIULIA,GIULIA,STELLANTIS,GIULIA 2.2 Diesel (190ch) AT8,GAZOLE,BERLINE,2143,SUPERIEURE,10,...,5.238000e+16,4.392000e+16,49005.0,5.106000e+03,-1.0,186195.0,137.26,1.150750e+16,128.41,133.8
2,ALFA ROMEO,GIULIA,GIULIA,STELLANTIS,GIULIA 2.0 T Essence (280ch) AT8,ESSENCE,BERLINE,1995,SUPERIEURE,17,...,7.698500e+04,6.311500e+16,6699.0,7.326000e+16,-1.0,244155.0,174435.00,1.430050e+05,151785.00,166.0
3,ALFA ROMEO,GIULIA,GIULIA,STELLANTIS,GIULIA 2.0 T Essence (280ch) AT8,ESSENCE,BERLINE,1995,SUPERIEURE,17,...,7.698500e+04,6.311500e+16,6699.0,7.326000e+16,-1.0,244155.0,174435.00,1.430050e+05,151785.00,166.0
4,ALFA ROMEO,GIULIA,GIULIA,STELLANTIS,GIULIA 2.2 Diesel (190ch) AT8,GAZOLE,BERLINE,2143,SUPERIEURE,10,...,5.238000e+16,4.392000e+16,49005.0,5.106000e+03,-1.0,186195.0,137.26,1.150750e+16,128.41,133.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6685,VOLVO,XC90,XC90,GEELY,XC90 B5 AWD 7PL (235ch) Geatronic 8,GAZ+ELEC HNR,TS TERRAINS/CHEMINS,1969,LUXE,13,...,7.062000e+03,6.185500e+15,72585.0,7.067000e+03,-1.0,225325.0,184865.00,1.618750e+05,190.03,185.0
6686,VOLVO,XC90,XC90,GEELY,XC90 B5 AWD 7PL (235ch) Geatronic 8,GAZ+ELEC HNR,TS TERRAINS/CHEMINS,1969,LUXE,13,...,7.062000e+03,6.185500e+15,72585.0,7.067000e+03,-1.0,225325.0,184865.00,1.618750e+05,190.03,185.0
6687,VOLVO,XC90,XC90,GEELY,XC90 B5 AWD 7PL (235ch) Geatronic 8,GAZ+ELEC HNR,TS TERRAINS/CHEMINS,1969,LUXE,13,...,7.062000e+03,6.185500e+15,72585.0,7.067000e+03,-1.0,225325.0,184865.00,1.618750e+05,190.03,185.0
6688,VOLVO,XC90,XC90,GEELY,XC90 B5 AWD 7PL (235ch) Geatronic 8,GAZ+ELEC HNR,TS TERRAINS/CHEMINS,1969,LUXE,13,...,7.062000e+03,6.185500e+15,72585.0,7.067000e+03,-1.0,225325.0,184865.00,1.618750e+05,190.03,185.0


In [149]:
# List of columns to keep
columns_to_keep = [
    "Marca", "Nombre del modelo", "Modelo", "Peso en vacío"
]

# Keeping only the specified columns
df_pesos2 = df_pesos[columns_to_keep].copy()

# Display the filtered dataframe
df_pesos2

,Marca,Nombre del modelo,Modelo,Peso en vacío
0,ALFA ROMEO,GIULIA,GIULIA,1429
1,ALFA ROMEO,GIULIA,GIULIA,1465
2,ALFA ROMEO,GIULIA,GIULIA,1429
3,ALFA ROMEO,GIULIA,GIULIA,1429
4,ALFA ROMEO,GIULIA,GIULIA,1465
...,...,...,...,...
6685,VOLVO,XC90,XC90,2095
6686,VOLVO,XC90,XC90,2095
6687,VOLVO,XC90,XC90,2095
6688,VOLVO,XC90,XC90,2095


In [150]:
# Calculate the average 'Peso en vacío' grouped by 'Marca' and 'Nombre del Modelo'
df_pesos2['Average weight'] = df_pesos2.groupby(['Marca', 'Modelo'])['Peso en vacío'].transform('mean')

# Display the updated DataFrame
df_pesos2


,Marca,Nombre del modelo,Modelo,Peso en vacío,Average weight
0,ALFA ROMEO,GIULIA,GIULIA,1429,1471.525
1,ALFA ROMEO,GIULIA,GIULIA,1465,1471.525
2,ALFA ROMEO,GIULIA,GIULIA,1429,1471.525
3,ALFA ROMEO,GIULIA,GIULIA,1429,1471.525
4,ALFA ROMEO,GIULIA,GIULIA,1465,1471.525
...,...,...,...,...,...
6685,VOLVO,XC90,XC90,2095,2095.000
6686,VOLVO,XC90,XC90,2095,2095.000
6687,VOLVO,XC90,XC90,2095,2095.000
6688,VOLVO,XC90,XC90,2095,2095.000


In [152]:
# Select one row for each unique combination of "Marca" and "Nombre del modelo"
df_pesos3 = df_pesos2.drop_duplicates(subset=['Marca', 'Modelo']).copy()

# Display the resulting DataFrame
df_pesos3


,Marca,Nombre del modelo,Modelo,Peso en vacío,Average weight
0,ALFA ROMEO,GIULIA,GIULIA,1429,1471.525000
40,ALFA ROMEO,STELVIO,STELVIO,1680,1702.446809
87,ALFA ROMEO,TONALE,TONALE,1525,1525.000000
95,ALPINE,A110,A110,1109,1105.562500
111,AUDI,Q2,Q2,1250,1335.000000
...,...,...,...,...,...
6604,VOLVO,V60,V60,1686,1733.350000
6624,VOLVO,V90,V90,1819,1819.000000
6625,VOLVO,XC40,XC40,1613,1593.878788
6658,VOLVO,XC60,XC60,1837,1855.833333


In [154]:
# Ensure df_maker_model2 and df_pesos3 are independent DataFrames
df_maker_model2 = df_maker_model2.copy()

# Define a helper function to handle the transformation
def standardize_column(value):
    if pd.isna(value) or str(value).strip().lower() == 'n/a':  # Check for NaN or 'N/A'
        return value  # Keep as is
    return unidecode(str(value)).lower()  # Convert to string, remove accents, and lowercase

# Standardize and clean the columns in df_maker_model2
df_maker_model2['Maker_final'] = df_maker_model2['Maker_final'].astype(str).str.strip().str.lower().apply(standardize_column)
df_maker_model2['Model_final'] = df_maker_model2['Model_final'].astype(str).str.strip().str.lower().apply(standardize_column)

# Standardize and clean the columns in df_pesos2
df_pesos3['Marca'] = df_pesos3['Marca'].astype(str).str.strip().str.lower().apply(standardize_column)
df_pesos3['Modelo'] = df_pesos3['Modelo'].astype(str).str.strip().str.lower().apply(standardize_column)

# Verify the standardized columns
print(df_maker_model2[['Maker_final', 'Model_final']].head())
print(df_pesos3[['Marca', 'Modelo']].head())

  Maker_final Model_final
0       honda      accord
1      nissan     estacas
2  volkswagen       vento
3        jeep    wrangler
4   chevrolet        beat
          Marca   Modelo
0    alfa romeo   giulia
40   alfa romeo  stelvio
87   alfa romeo   tonale
95       alpine     a110
111        audi       q2


In [155]:
df_maker_model2

,numcaso,tipo_vehiculo,fecha_servicio,plate,model,Tipo de Auto,car_type,clean_serie,Maker_final,Model_final
0,355845,Auto,2023-09-01 00:01:13,SZD861A,2003,estandar,Auto,3HGCM66563G001615,honda,accord
1,355847,Camion,2023-09-01 00:06:09,TP9498G,2014,estandar,camion,3N6DD25T2EK005444,nissan,estacas
2,355849,Auto,2023-09-01 00:08:38,NCZ1892,2018,estandar,Auto,MEX5G2603JT120368,volkswagen,vento
3,355857,Auto,2023-09-01 00:24:34,FCJ597B,2015,estandar,Auto,1C4BJWEG6FL629655,jeep,wrangler
4,355864,Auto,2023-09-01 00:46:09,NJA2747,2019,estandar,Auto,MA6CB6CD6KT029772,chevrolet,beat
...,...,...,...,...,...,...,...,...,...,...
39995,430448,Camion,2023-11-04 14:22:12,JX40237,2021,estandar,camion,3NFAB25R4MM000066,hyundai,cargo van
39996,430450,Auto,2023-11-04 14:23:14,JNA3243,2017,estandar,Auto,9BWDB45U5HT015012,volkswagen,gol
39997,430453,Auto,2023-11-04 14:25:29,TMH6032,2019,estandar,Auto,WAUAYJ8V6KA077601,audi,a3
39998,430455,Auto,2023-11-04 14:26:07,JSH4002,2021,estandar,Auto,TMCJ33AB4MJ157474,n/a caso 3,tucson


In [156]:
df_pesos3

,Marca,Nombre del modelo,Modelo,Peso en vacío,Average weight
0,alfa romeo,GIULIA,giulia,1429,1471.525000
40,alfa romeo,STELVIO,stelvio,1680,1702.446809
87,alfa romeo,TONALE,tonale,1525,1525.000000
95,alpine,A110,a110,1109,1105.562500
111,audi,Q2,q2,1250,1335.000000
...,...,...,...,...,...
6604,volvo,V60,v60,1686,1733.350000
6624,volvo,V90,v90,1819,1819.000000
6625,volvo,XC40,xc40,1613,1593.878788
6658,volvo,XC60,xc60,1837,1855.833333


In [157]:
# Export the DataFrame to an Excel file
df_maker_model2.to_excel("maker_model2.xlsx", index=False)

df_pesos3.to_excel("pesos3.xlsx", index=False)

# Verify the file path (useful for Jupyter/Colab environments)
print("DataFrame has been exported to 'output_file.xlsx'")


DataFrame has been exported to 'output_file.xlsx'


In [158]:
# Find rows in df_maker_model2 that don't match df_pesos2
unmatched = df_maker_model2.merge(
    df_pesos3[['Marca', 'Modelo']],
    how='left',
    left_on=['Maker_final', 'Model_final'],
    right_on=['Marca', 'Modelo'],
    indicator=True
)

# Display unmatched rows
print(unmatched[unmatched['_merge'] == 'left_only'])


       numcaso tipo_vehiculo      fecha_servicio    plate  model Tipo de Auto  \
0       355845          Auto 2023-09-01 00:01:13  SZD861A   2003     estandar   
1       355847        Camion 2023-09-01 00:06:09  TP9498G   2014     estandar   
2       355849          Auto 2023-09-01 00:08:38  NCZ1892   2018     estandar   
3       355857          Auto 2023-09-01 00:24:34  FCJ597B   2015     estandar   
4       355864          Auto 2023-09-01 00:46:09  NJA2747   2019     estandar   
...        ...           ...                 ...      ...    ...          ...   
39995   430448        Camion 2023-11-04 14:22:12  JX40237   2021     estandar   
39996   430450          Auto 2023-11-04 14:23:14  JNA3243   2017     estandar   
39997   430453          Auto 2023-11-04 14:25:29  TMH6032   2019     estandar   
39998   430455          Auto 2023-11-04 14:26:07  JSH4002   2021     estandar   
39999   430460        Camion 2023-11-04 14:29:39  SP60496   2021     estandar   

      car_type        clean

In [159]:
# Perform the merge operation
df_combined = df_maker_model2.merge(
    df_pesos3[['Marca', 'Modelo', 'Average weight']],  # Select relevant columns from df_pesos2
    how='left',                                       # Left join to retain all rows from df_maker_model2
    left_on=['Maker_final', 'Model_final'],           # Match these columns in df_maker_model2
    right_on=['Marca', 'Modelo']                      # Match these columns in df_pesos2
)

# # Fill missing values in "Average weight" with "N/A"
df_combined['Average weight'] = df_combined['Average weight'].fillna('N/A')

In [160]:
df_combined

,numcaso,tipo_vehiculo,fecha_servicio,plate,model,Tipo de Auto,car_type,clean_serie,Maker_final,Model_final,Marca,Modelo,Average weight
0,355845,Auto,2023-09-01 00:01:13,SZD861A,2003,estandar,Auto,3HGCM66563G001615,honda,accord,NaN,NaN,N/A
1,355847,Camion,2023-09-01 00:06:09,TP9498G,2014,estandar,camion,3N6DD25T2EK005444,nissan,estacas,NaN,NaN,N/A
2,355849,Auto,2023-09-01 00:08:38,NCZ1892,2018,estandar,Auto,MEX5G2603JT120368,volkswagen,vento,NaN,NaN,N/A
3,355857,Auto,2023-09-01 00:24:34,FCJ597B,2015,estandar,Auto,1C4BJWEG6FL629655,jeep,wrangler,NaN,NaN,N/A
4,355864,Auto,2023-09-01 00:46:09,NJA2747,2019,estandar,Auto,MA6CB6CD6KT029772,chevrolet,beat,NaN,NaN,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,430448,Camion,2023-11-04 14:22:12,JX40237,2021,estandar,camion,3NFAB25R4MM000066,hyundai,cargo van,NaN,NaN,N/A
39996,430450,Auto,2023-11-04 14:23:14,JNA3243,2017,estandar,Auto,9BWDB45U5HT015012,volkswagen,gol,NaN,NaN,N/A
39997,430453,Auto,2023-11-04 14:25:29,TMH6032,2019,estandar,Auto,WAUAYJ8V6KA077601,audi,a3,NaN,NaN,N/A
39998,430455,Auto,2023-11-04 14:26:07,JSH4002,2021,estandar,Auto,TMCJ33AB4MJ157474,n/a caso 3,tucson,NaN,NaN,N/A


In [161]:
df_combined.to_excel("combined.xlsx", index=False)